In [1]:
import pyedflib # ref: https://pyedflib.readthedocs.io/en/latest/
import pandas as pd
import numpy as np
import os
from datetime import timedelta
import matplotlib.pyplot as plt
from scipy import signal
import fnmatch

In [2]:
sth = "EOG"
threshold = 30

read_folder = './clips/'
write_folder = f'./plots_n1_{threshold}_{sth}/'
os.makedirs(write_folder, exist_ok=True)
pattern1 = '*-PSG.edf'
pattern2 = '*-Hypnogram.edf'

psg_list = sorted([f for f in os.listdir(read_folder) if fnmatch.fnmatch(f, pattern1)])
hypnogram_list = sorted([f for f in os.listdir(read_folder) if fnmatch.fnmatch(f, pattern2)])
psg_iter = iter(psg_list)
hypnogram_iter = iter(hypnogram_list)

channels = ['EEG Fpz-Cz', 'EEG Pz-Oz']
channel = channels[1]
psg_id = None
hypnogram_id = None

In [3]:
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import matplotlib.dates as mdates
import pandas as pd
def lol(signal_df):
    num_columns = len(signal_df.columns)

    fig, axes = plt.subplots(num_columns, 1, figsize=(20, 12), sharex=True)

    # 迴圈遍歷每個欄位
    for i, column in enumerate(signal_df.columns):
        # 取得目前的軸
        ax = axes[i]

        # 繪製折線圖
        ax.plot(signal_df.index, signal_df[column])
        
        # 繪製虛線
        start_time = signal_df.index[0]
        end_time = signal_df.index[-1]
        interval = pd.Timedelta(seconds=30)
        current_time = start_time + interval
        while current_time < end_time:
            ax.axvline(x=current_time, linestyle='--', color='gray')
            current_time += interval

        # 設定軸的標籤
        ax.set_ylabel(column)
        loc = mdates.MinuteLocator(interval=1)
        ax.xaxis.set_major_locator(loc)
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

    # 設定圖表標題和共用 x 軸標籤
    fig.suptitle('Signal Visualization')
    axes[-1].set_xlabel('Time')

    # 調整子圖之間的間距
    plt.tight_layout()
    # 顯示圖表
    # plt.show()
    plt.savefig(write_folder + psg_id.split('.')[0],bbox_inches='tight')


In [4]:
def get_n1_index(signal_df, threshold=25):
    index = []
    index_series = []
    # 0, 3000, 6000
    N = signal_df.shape[0]
    # # FIXED VARIABLE
    # threshold = 25

    for i in range(0, N, 1500):
        mxcount = 0
        count = 0
        try:
            for j in range(3000):
                if(abs(signal_df.iloc[i+j,0]) < threshold and abs(signal_df.iloc[i+j-1,0]) < threshold):
                    count += 1
                else:
                    if(count > mxcount):
                        mxcount = count
                    count = 0
            mxcount = max(mxcount, count)
        except:
            pass
        # print(i,mxcount)
        index_series.append(mxcount)
        if(mxcount > 1500):
            index.append(i)
    # len(index)
    return index

In [5]:
def eog_n1_predict(signal_df, window = 5, ratio = 0.5):
    
    # window = moving average window
    # ratio = if max value * ratio < all seen values in the interval, then N1
    index_eog = []
    N = signal_df.shape[0]
    # FIXED VARIABLE
    sum_max = 0
    seen = 0 # how many intervals passed


    for i in range(0, N, 3000):
        max_len = min(i+3000, N)
        cur_max = signal_df.iloc[i:max_len].max()
        if seen > window:
            avg = (sum_max / seen)
            if avg * ratio > cur_max:
                print("FOUND")
                return [i]
            else:
                print(f"cur_max: {cur_max}, avg: {avg}")
        sum_max += cur_max
        seen += 1
    

In [6]:
gcount = 0
zero_count = 0
while True:
    gcount += 1
    global psg_id, hypnogram_id
    try:
        psg_id = next(psg_iter)
        hypnogram_id = next(hypnogram_iter)
    except StopIteration:
        print("Iter Stop")
        break
    signal_path = os.path.join(read_folder, f'{psg_id}')
    label_path = os.path.join(read_folder, f'{hypnogram_id}')
    # read EDF files
    if 'edf_label' and 'edf_label' in globals():
        del edf_signal, edf_label
    edf_signal = pyedflib.EdfReader(signal_path)
    edf_label = pyedflib.EdfReader(label_path)

    # read data
    annotations = edf_label.readAnnotations()
    start = edf_signal.getStartdatetime()
    signals, frequencies = edf_signal.getSignalLabels(), edf_signal.getSampleFrequencies()
    data = []
    for ch_idx, sig_name, freq in zip( range(len(signals)), signals, frequencies,):
        sig = edf_signal.readSignal(chn=ch_idx)
        idx = pd.date_range(  start=start, periods=len(sig), freq=pd.Timedelta(1 / freq, unit="s") )
        data += [pd.Series(sig, index=idx, name=sig_name)]
    # create DataFrames
    annotations_df = pd.DataFrame(annotations)
    annotations_df = annotations_df.T
    annotations_df.rename(columns={0: "Onset", 1: "Duration", 2:"Annotations"}, inplace=True)
    signal_df =pd.concat(data[0:3], axis=1)
    signal_df['N1'] = 0

    def check_sleep_stage(row):
        start_time = start + timedelta(seconds = int(annotations_df['Onset'].iloc[1]))
        end_time = start + timedelta(seconds = int(annotations_df['Onset'].iloc[1])) + timedelta(seconds = int(annotations_df['Duration'].iloc[1]))
        if start_time <= pd.to_datetime(row.name) < end_time:
            return int(1)
        else:
            return int(0)

    signal_df['N1'] = signal_df.apply(check_sleep_stage, axis=1)
    signal_df['N1_predict_EEG'] = 0


    def check_sleep_stage_predict(row, start_index, duration):
        start_time = start + timedelta(seconds = start_index)
        end_time = start + timedelta(seconds = start_index) + timedelta(seconds = duration)
        if start_time <= pd.to_datetime(row.name) < end_time:
            return int(1)
        else:
            return int(0)
    index = get_n1_index(signal_df, threshold=threshold)
    val = eog_n1_predict(signal_df['EOG horizontal'], window=5, ratio=0.5)
    if len(index) == 0:
        # zero_count += 1
        # print(psg_id)
        pass
    elif len(index) == 1:
        signal_df['N1_predict_EEG'] = signal_df.apply(check_sleep_stage_predict, args=(int(index[0]/100), int(max(index)/100) + 1.5), axis=1) 
    elif len(index) == 2:
        signal_df['N1_predict_EEG'] = signal_df.apply(check_sleep_stage_predict, args=(int(index[0]/100), int(max(index)/100)), axis=1)
    else:
        signal_df['N1_predict_EEG'] = signal_df.apply(check_sleep_stage_predict, args=(int(index[0]/100), int((max(index)-index[0])/100)), axis=1)    

    if len(val) == 0:
        pass
    elif len(val) == 1:
        signal_df['N1_predict_EOG'] = signal_df.apply(check_sleep_stage_predict, args=(int(val[0]/100), int(max(val)/100) + 1.5), axis=1) 

    if (signal_df.get('N1_predict_EEG') is not None and signal_df.get('N1_predict_EOG') is not None):
        signal_df['N1_predict'] = (signal_df['N1_predict_EEG'] & signal_df['N1_predict_EOG'])
        if len(signal_df['N1_predict']) ==0:
            signal_df['N1_predict'] = signal_df['N1_predict_EOG']
    elif signal_df.get('N1_predict_EEG') is not None:
        signal_df['N1_predict'] = signal_df['N1_predict_EEG']
    elif signal_df.get('N1_predict_EOG') is not None:
        signal_df['N1_predict'] = signal_df['N1_predict_EOG']
    signal_df['N1_predict'] = signal_df['N1_predict'].astype('bool')

    lol(pd.concat([signal_df], axis=1))
    # print(f"zero_count/total: {zero_count}/{gcount}", end='\r')
# print(f"zero_count/total: {zero_count}/{gcount}")

cur_max: 330.41978021978025, avg: 285.82173382173386
cur_max: 287.05372405372407, avg: 292.19288330716904
cur_max: 335.84053724053723, avg: 291.5504884004884
cur_max: 441.2989010989011, avg: 296.47160493827164
cur_max: 437.8493284493285, avg: 310.9543345543346
FOUND
FOUND
cur_max: 380.63736263736274, avg: 350.94566544566555
cur_max: 309.3772893772895, avg: 355.1873364730509
cur_max: 261.20146520146534, avg: 349.4610805860807
cur_max: 268.72893772893786, avg: 339.6544566544568
cur_max: 408.23809523809535, avg: 332.5619047619049
cur_max: 396.6959706959708, avg: 339.4415584415586
cur_max: 357.05128205128216, avg: 344.2127594627596
cur_max: 271.73992673992683, avg: 345.20033812341524
cur_max: 214.02930402930414, avg: 339.95316588173745
FOUND
cur_max: 408.83809523809526, avg: 311.6736263736264
cur_max: 435.0747252747253, avg: 325.55426478283624
cur_max: 428.8490842490843, avg: 339.2443223443224
cur_max: 292.774358974359, avg: 349.20040700040704
FOUND
cur_max: 443.53846153846155, avg: 345.65

/tmp/ipykernel_535647/250010612.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axes = plt.subplots(num_columns, 1, figsize=(20, 12), sharex=True)


cur_max: 206.8100122100122, avg: 355.7492877492877
FOUND
cur_max: 230.94285714285724, avg: 135.79523809523818
cur_max: 187.02857142857152, avg: 149.3877551020409
cur_max: 92.13333333333343, avg: 154.0928571428572
cur_max: 206.2095238095239, avg: 147.20846560846567
cur_max: 152.20000000000007, avg: 153.1085714285715
cur_max: 172.39047619047628, avg: 153.0259740259741
cur_max: 139.58095238095245, avg: 154.6396825396826
cur_max: 218.8285714285715, avg: 153.48131868131875
cur_max: 102.73333333333342, avg: 158.14897959183682
cur_max: 135.03809523809534, avg: 154.45460317460325
cur_max: 105.25714285714294, avg: 153.2410714285715
cur_max: 148.66666666666674, avg: 150.41848739495805
cur_max: 205.20000000000007, avg: 150.3211640211641
cur_max: 195.104761904762, avg: 153.2095238095239
cur_max: 178.95238095238105, avg: 155.3042857142858
cur_max: 201.66666666666674, avg: 156.43038548752844
cur_max: 181.47619047619057, avg: 158.4865800865802
cur_max: 166.83809523809532, avg: 159.48612836438934
FOUN

In [ ]:
# # 取得資料的欄位數量

# # 設定圖表大小
# signal_df_new = pd.concat([signal_df, data[2]], axis=1)
# lol(signal_df_new)